<a href="https://colab.research.google.com/github/andysingal/transfer-learning/blob/main/NLP_sentimentanalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi




Thu Jun  3 02:22:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from __future__ import absolute_import,division,print_function,unicode_literals
try:
  !pip uninstall tb-nightly tensorboardX tensorboard
  !pip install tf-nightly
except Exception:
     pass
import tensorflow as tf
import pandas as pd
import numpy as np

import os
import datetime
import tensorflow_datasets as tfds

%load_ext tensorboard


Uninstalling tensorboard-2.5.0:
  Would remove:
    /usr/local/bin/tensorboard
    /usr/local/lib/python3.7/dist-packages/tensorboard-2.5.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorboard/*
Proceed (y/n)? y
  Successfully uninstalled tensorboard-2.5.0
     |████████████████████████████████| 451.4MB 37kB/s 
     |████████████████████████████████| 5.9MB 49.7MB/s 
     |████████████████████████████████| 471kB 41.1MB/s 
     |████████████████████████████████| 4.2MB 23.8MB/s 
     |████████████████████████████████| 1.3MB 44.1MB/s 
ERROR: tensorflow 2.5.0 requires tensorboard~=2.5, which is not installed.
ERROR: tensorflow 2.5.0 has requirement grpcio~=1.34.0, but you'll have grpcio 1.38.0 which is incompatible.
ERROR: tensorflow 2.5.0 has requirement keras-nightly~=2.5.0.dev, but you'll have keras-nightly 2.6.0.dev2021052700 which is incompatible.
  Found existing installation: grpcio 1.34.1
    Uninstalling grpcio-1.34.1:
      Successfully uninstalled grpcio-1.34.1
  Fo

In [3]:
import pkg_resources
for entry_point in pkg_resources.iter_entry_points('tensorflow_plugins'):
  print(entry_point.dist)

In [4]:
#!ls -alrt /usr/local/lib/python3.7/dist-packages/tensorboard*
!rm -r  /usr/local/lib/python3.7/dist-packages/tensorboardcolab-0.22.dist-info


rm: cannot remove '/usr/local/lib/python3.7/dist-packages/tensorboardcolab-0.22.dist-info': No such file or directory


In [5]:
print(tf.__version__)

2.6.0-dev20210602


In [6]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [16]:
dataset,info = tfds.load('amazon_us_reviews/Personal_Care_Appliances_v1_00', with_info=True)
train_dataset = dataset['train']

In [17]:
info

tfds.core.DatasetInfo(
    name='amazon_us_reviews',
    version=0.1.0,
    description='Amazon Customer Reviews (a.k.a. Product Reviews) is one of Amazons iconic products. In a period of over two decades since the first review in 1995, millions of Amazon customers have contributed over a hundred million reviews to express opinions and describe their experiences regarding products on the Amazon.com website. This makes Amazon Customer Reviews a rich source of information for academic researchers in the fields of Natural Language Processing (NLP), Information Retrieval (IR), and Machine Learning (ML), amongst others. Accordingly, we are releasing this data to further research in multiple disciplines related to understanding customer product experiences. Specifically, this dataset was constructed to represent a sample of customer evaluations and opinions, variation in the perception of a product across geographical regions, and promotional intent or bias in reviews.

Over 130+ million cus

In [21]:
BUFFER_SIZE=30000
BATCH_SIZE=128

In [22]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE,reshuffle_each_iteration=False)

In [24]:
for reviews in train_dataset.take(2):
  print(reviews)

{'data': {'customer_id': <tf.Tensor: shape=(), dtype=string, numpy=b'21853030'>, 'helpful_votes': <tf.Tensor: shape=(), dtype=int32, numpy=1>, 'marketplace': <tf.Tensor: shape=(), dtype=string, numpy=b'US'>, 'product_category': <tf.Tensor: shape=(), dtype=string, numpy=b'Personal_Care_Appliances'>, 'product_id': <tf.Tensor: shape=(), dtype=string, numpy=b'B000FKJJYI'>, 'product_parent': <tf.Tensor: shape=(), dtype=string, numpy=b'927909910'>, 'product_title': <tf.Tensor: shape=(), dtype=string, numpy=b"Dr. Scholl's For Her Open Shoe Insoles with Massaging Gel, Women's Sizes 6-10, 1 pair (Pack of 3)">, 'review_body': <tf.Tensor: shape=(), dtype=string, numpy=b'I like these insoles because they prevent my feet from slipping in my shoes and they are so flat that they do not take up much space like some other insoles.'>, 'review_date': <tf.Tensor: shape=(), dtype=string, numpy=b'2007-03-13'>, 'review_headline': <tf.Tensor: shape=(), dtype=string, numpy=b'Comfy and flat'>, 'review_id': <tf.

In [25]:
for reviews in train_dataset.take(10):
   review_text = reviews['data']
   print(review_text.get('review_body').numpy())
   print(review_text.get('star_rating'))
   print(tf.where(review_text.get('star_rating') >3,1,0).numpy())

b'I like these insoles because they prevent my feet from slipping in my shoes and they are so flat that they do not take up much space like some other insoles.'
tf.Tensor(3, shape=(), dtype=int32)
0
b"I have used this product for years and the only place I could find it was in a drugstore.  The cost was a bit high but I only realized recently that perhaps it was available through Amazon at a better price and quantity.      I bought 6 boxes and now I know that when I have back problems I can use Bodi Heat.  It is simply the best thing I've found.  You can put it on in the morning and it lasts well into the evening.  It stays attached until you take it off and it provides more than enough heat.  It not only soothes the lower back but I've found that it also eases the muscle pain in the upper back.  It is certainly better than ThermaCare.    You can also use this item to simply keep warm in winter or to help ease muscle pain from the flu or cold.  I am so pleased to find it online and I w

Building a vocabulary, tokenising and encoding.
First, build a vocabulary by tokenizing the text into a collection of individual unique words. There are a few ways to do this in both TensorFlow and Python. For this tutorial:
Iterate over each example’s numpy value.
Use tfds.features.text.Tokenizer to split it into tokens.
Collect these tokens into a Python set, to remove duplicates.
Get the size of the vocabulary for later use.

In [27]:
tokenizer = tfds.deprecated.text.Tokenizer()

vocabulary_set = set()
for _,reviews in train_dataset.enumerate():
  review_text = reviews['data']
  review_tokens = tokenizer.tokenize(review_text.get('review_body').numpy())
  vocabulary_set.update(review_tokens)

vocab_size = len(vocabulary_set)
vocab_size

68782

Encode examples
Create an encoder by passing the vocabulary_set to tfds.features.text.TokenTextEncoder. The encoder's encode method takes in a string of text and returns a list of integers.

https://alexmoltzau.medium.com/building-a-text-dataset-c5c1481395f4


In [29]:
encoder = tfds.deprecated.text.TokenTextEncoder(vocabulary_set)

In [30]:
print(vocabulary_set)

{'crappiest', 'Blurry', 'dailiy', 'ION', 'modifies', 'briefer', 'relieved', 'Jatai', 'En', 'hieght', 'astute', 'straightoner', 'Wiesbaden', '48lb', 'Infinity', 'RTFM', 'Deffinitely', 'sdfgvj', 'encumbered', 'netted', 'blogged', 'polycarbonate', 'revue', 'everyoen', 'Satan', 'Develops', '6º', 'hypotension', 'OPD', 'dumping', 'iffffy', 'Chokes', 'handbrakes', 'unexperienced', 'response', 'ultrasoft', 'H', 'fitwatch', 'facia', 'stub', 'arguably', 'CALL', 'skeem', 'discounted_price', 'Merlot', 'mDrive', 'Kitten', 'Puss', 'frying', 'draft', 'Constant', 'stanch', 'Velcros', 'crummy', 'arefund', 'Rofe', 'perpetuate', 'Door', 'slicker', 'losses', 'plateau', 'Trooper', 'technophobe', 'genre', 'bagels', 'picstiks', 'eyeteeth', 'MaryLeeParker', 'glucosimine', 'tnat', 'shinier', 'Sherman', 'whoa', 'sidebar', 'scald', 'compulsively', 'aggrevation', 'induction', 'misstatements', 'SAF', 'splintered', 'infusserthat', 'KEEVA', '490', 'Afromosia', 'matresses', 'contoures', 'SLIMMER', 'DESTROY', 'GRIND',

In [32]:
for reviews in train_dataset.take(10):
   review_text = reviews['data']
   print(review_text.get('review_body').numpy())
   encode_example = encoder.encode(review_text.get('review_body').numpy())
   print(encode_example)

b'I like these insoles because they prevent my feet from slipping in my shoes and they are so flat that they do not take up much space like some other insoles.'
[44130, 39596, 39735, 13966, 59765, 31430, 63264, 15293, 3932, 61950, 34564, 36411, 15293, 62619, 424, 31430, 9313, 58912, 25847, 16784, 31430, 64937, 53255, 51630, 45018, 15361, 43179, 39596, 19472, 5244, 13966]
b"I have used this product for years and the only place I could find it was in a drugstore.  The cost was a bit high but I only realized recently that perhaps it was available through Amazon at a better price and quantity.      I bought 6 boxes and now I know that when I have back problems I can use Bodi Heat.  It is simply the best thing I've found.  You can put it on in the morning and it lasts well into the evening.  It stays attached until you take it off and it provides more than enough heat.  It not only soothes the lower back but I've found that it also eases the muscle pain in the upper back.  It is certainly b

In [33]:
for index in encode_example:
  print('{} ----> {}'.format(index,encoder.decode([index])))

35527 ----> Bought
39735 ----> these
23283 ----> on
10188 ----> the
16729 ----> recommendation
25380 ----> of
10188 ----> the
45330 ----> Skin
63640 ----> Care
30128 ----> Addiction
40963 ----> subreddit
23286 ----> http
14257 ----> www
21883 ----> reddit
41328 ----> com
4607 ----> r
24210 ----> skincareaddiction
44130 ----> I
22594 ----> cut
10188 ----> the
8661 ----> bandage
66798 ----> into
28876 ----> small
1991 ----> pieces
56614 ----> about
59817 ----> quarter
6919 ----> sized
424 ----> and
16123 ----> applied
31895 ----> one
61617 ----> to
15293 ----> my
35207 ----> skin
52575 ----> where
46087 ----> a
51573 ----> pimple
28591 ----> was
12999 ----> forming
34607 ----> Overnight
10188 ----> the
8661 ----> bandage
8439 ----> helped
49568 ----> bring
10188 ----> the
20848 ----> infected
26510 ----> part
61617 ----> to
10188 ----> the
55363 ----> surface
424 ----> and
8439 ----> helped
61269 ----> it
23578 ----> heal
15361 ----> much
64857 ----> faster
34333 ----> than
51202 ----> n

In [53]:
def encode(text_sensor,label):
  encoded_text = encoder.encode(text_sensor.numpy())
  label = tf.where(label >3,1,0)
  return encoded_text,label

In [54]:
def encode_map_fn(tensor):
  text = tensor['data'].get('review_body')
  label = tensor['data'].get('star_rating')
  # py_func doesn't set the shape of the returned tensors.
  encoded_text, label = tf.py_function(encode, 
                                       inp=[text, label], 
                                       Tout=(tf.int64, tf.int32))

  # `tf.data.Datasets` work best if all components have a shape set
  #  so set the shapes manually: 
  encoded_text.set_shape([None])
  label.set_shape([])

  return encoded_text, label

https://notebook.community/tensorflow/docs/site/en/tutorials/load_data/text

In [55]:
all_encoded_data = train_dataset.map(encode_map_fn)

In [56]:
for f0,f1 in all_encoded_data.take(2):
  print(f0)
  print(f1)

tf.Tensor(
[44130 39596 39735 13966 59765 31430 63264 15293  3932 61950 34564 36411
 15293 62619   424 31430  9313 58912 25847 16784 31430 64937 53255 51630
 45018 15361 43179 39596 19472  5244 13966], shape=(31,), dtype=int64)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(
[44130 11380 18543 41726 51974 23807  6434   424 10188 22779 62414 44130
  1454 65437 61269 28591 36411 46087 25446 63117 40198 28591 46087  7854
 24245 53229 44130 22779  9001 65171 16784  5331 61269 28591 22876  1415
 27018 58357 46087 47465 17550   424 48957 44130 63253 51855 62651   424
 60882 44130 17820 16784 60361 44130 11380 45062  2850 44130 32496 54784
 26606 23998 30166 46605 52392 10188 14816 16193 44130  6490 43984  8188
 32496 14382 61269 23283 36411 10188 14988   424 61269 53009 63210 66798
 10188 44842 30166  5112 38386  1767 22555 51630 61269 54999   424 61269
 63017 68684 34333 53715  4599 30166 53255 22779 10652 10188 58602 45062
 53229 44130  6490 43984 16784 61269 11958  2551 10188 52616  2150 3

https://notebook.community/tensorflow/docs/site/en/tutorials/load_data/text

In [57]:
TAKE_SIZE = 10000
train_data = all_encoded_data.skip(TAKE_SIZE).shuffle(BUFFER_SIZE)
train_data = train_data.padded_batch(BATCH_SIZE)

test_data = all_encoded_data.take(TAKE_SIZE)
test_data = test_data.padded_batch(BATCH_SIZE)

Since we have introduced a new token encoding (the zero used for padding), the vocabulary size has increased by one.

In [58]:
vocab_size += 1

BUILDING THE MODEL

In [11]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, 128))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128,return_sequences=True)))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))

